<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#思路" data-toc-modified-id="思路-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>思路</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#attribute-selection" data-toc-modified-id="attribute-selection-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>attribute selection</a></span></li><li><span><a href="#Outliers" data-toc-modified-id="Outliers-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Outliers</a></span></li></ul></li></ul></div>

# 思路
- 预处理 怎么找到词库?

In [1]:
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn import mixture
from sklearn.cluster import KMeans
from nltk.cluster import KMeansClusterer, cosine_distance
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support, classification_report, roc_curve, auc, precision_recall_curve
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
seeds = 1234

In [3]:
train = pd.read_json('../data/structured_train.json')
test = pd.read_json('../data/structured_test.json')

l = train['long_string'].dropna().apply(len).sort_values()
longest_idx = l[l>0].index.tolist()[-30:]
train = train[~train.global_index.isin(longest_idx)].reset_index(drop=True)
# train['long_string'].sample(10).tolist()

In [4]:
# train = train.groupby('label').sample(50, random_state=seeds)
# test = test.groupby('label').sample(50, random_state=seeds)

# EDA

## attribute selection

In [5]:
t = train.isnull().sum().sort_values()
not_used_cols = t[t > train.shape[0]*0.1].index.tolist()
print("not_used_cols: \n", not_used_cols)
t.to_frame().style

not_used_cols: 
 ['Distribution', 'Nntp-Posting-Host', 'NNTP-Posting-Host', 'Reply-To', 'Keywords', 'Article-I.D.', 'X-Newsreader', 'Summary', 'Originator', 'In-Reply-To', 'News-Software', 'Expires', 'In-reply-to', 'To', 'X-Disclaimer', 'Disclaimer', 'X-Mailer', 'X-Useragent', 'X-Xxdate', 'X-UserAgent', 'Supersedes', 'X-XXDate', 'Nf-ID', 'X-News-Reader', 'Nf-From', 'X-Added', 'Original-Sender', 'X-Posted-From', 'X-XXMessage-ID', 'Content-Type', 'X-Xxmessage-Id', 'Cc', 'X-Last-Updated', 'Mime-Version', 'Moderator', 'Content-Transfer-Encoding', 'date', 'X-Gated-By', 'X-To', 'Return-Path', 'X-Received', 'X-Sender', 'Nntp-Posting-User', 'X-Auth-User', 'X-X-From', 'X-Md4-Signature', 'X-Maildoor', 'X-Cc', 'X-NewsSoftware', 'X-AltNet-ID', 'MIME-Version', 'Return-Receipt-To', 'X-Header', 'X-Newssoftware', 'Comments', 'Bcc', 'X-Bytes', 'X-Software', 'X-Copyright', 'Posting-Front-End', 'X-Mail-Reader', 'X-News-Software', 'X-Reader', 'Organisation', 'Apparently-To', 'X-Comment-To', 'Mmdf-Warning'

,0
global_index,0
tag_reply,0
delivered_to,0
reference_two,0
reference_one,0
reply,0
tag_reference_one,0
tag_reference_two,0
doc_path,0
original_idx,0


In [6]:
select_cols = ["global_index", "doc_path", "label", "reply", "reference_one", "reference_two",
               "Subject", "From", "Lines", "Organization", "contained_emails", "long_string", "text", "error_message"]
print("\nmay use cols: \n", select_cols)
train = train[select_cols]
test = test[select_cols]


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


In [7]:
train.head()

,global_index,doc_path,label,reply,reference_one,reference_two,Subject,From,Lines,Organization,contained_emails,long_string,text,error_message
0,0,../../data/train\alt.atheism\49960,alt.atheism,Archive-name: atheism/resources Alt-atheism-ar...,,,Alt.Atheism FAQ: Atheist Resources,mathew <mathew@mantis.co.uk>,290,"Mantis Consultants, Cambridge. UK.","[ <figmo@netcom.com>. , archive-server@mantis...",[],From: mathew <mathew@mantis.co.uk>\nSubject: A...,NaN
1,1,../../data/train\alt.atheism\51060,alt.atheism,Archive-name: atheism/introduction Alt-atheism...,,,Alt.Atheism FAQ: Introduction to Atheism,mathew <mathew@mantis.co.uk>,646,"Mantis Consultants, Cambridge. UK.","[ <mathew@mantis.co.uk> , pgpinfo@mantis.co.u...",[iQCVAgUBK8AjRXzXN+VrOblFAQFSbwP+MHePY4g7ge8Mo...,From: mathew <mathew@mantis.co.uk>\nSubject: A...,NaN
2,2,../../data/train\alt.atheism\51119,alt.atheism,The argument goes as follows: Q-oid quotes app...,This is a new argument to me. Could you elabor...,"Well, John has a quite different, not necessar...",Re: Gospel Dating,I3150101@dbstu1.rz.tu-bs.de (Benedikt Rosenau),93,"Technical University Braunschweig, Germany","[ <65974@mimsy.umd.edu> , mangoe@cs.umd.edu ]",[],From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...,NaN
3,3,../../data/train\alt.atheism\51120,alt.atheism,It sounds to me like it's just SCREAMING OUT f...,"Recently, RAs have been ordered to post a reli...",,Re: university violating separation of church/...,mathew <mathew@mantis.co.uk>,29,"Mantis Consultants, Cambridge. UK.",[dmn@kepler.unh.edu ],[],From: mathew <mathew@mantis.co.uk>\nSubject: R...,NaN
4,4,../../data/train\alt.atheism\51121,alt.atheism,Can somebody reconcile the apparent contradict...,"HOWEVER, I hate economic terrorism and politic...",,"Re: [soc.motss, et al.] ""Princeton axes matchi...",strom@Watson.Ibm.Com (Rob Strom),15,IBM Research,"[ <N4HY.93Apr5120934@harder.ccr-p.ida.org>, , ...",[],From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...,NaN


In [8]:
train.isnull().sum().sort_values().to_dict()

{'global_index': 0,
 'doc_path': 0,
 'label': 0,
 'reply': 0,
 'reference_one': 0,
 'reference_two': 0,
 'text': 0,
 'Subject': 11,
 'From': 11,
 'contained_emails': 11,
 'long_string': 11,
 'Lines': 48,
 'Organization': 491,
 'error_message': 11053}

In [9]:
# train[["label","global_index", "reply", "reference_one", "reference_two",]].sample(10).style

## Outliers

In [22]:
a

0        0
1        3
2        0
3        0
4        0
        ..
11048    0
11049    0
11050    0
11051    0
11052    0
Name: long_string, Length: 11053, dtype: int64

In [23]:
c

764      11
878      10
1607      7
2051      6
2147      6
2205      7
2811      5
2850      7
3018      6
3411      9
3698      5
3699      5
4979      6
5180      5
5835      6
5856      8
7004      5
7603      9
7604      9
7741      5
7762      6
7918      8
8280      6
8628      8
9168      5
10338     5
10354     5
Name: long_string, dtype: int64

In [ ]:
train

In [30]:
train['long_string_len'] = train['long_string'].fillna(value='').apply(lambda x: len(x))
train.sort_values(by=['long_string_len']).tail(50)

,global_index,doc_path,label,reply,reference_one,reference_two,Subject,From,Lines,Organization,contained_emails,long_string,text,error_message,long_string_len
1579,1583,../../data/train\comp.os.ms-windows.misc\9928,comp.os.ms-windows.misc,I have been experiencing several end-user prob...,,,workaround for Citizen drivers,mike@drd.com (Mike.Rovak),38,DRD Corporation,[ mike.rovak@drd.com ],[---------------------------------------------...,From: mike@drd.com (Mike.Rovak)\nSubject: work...,NaN,4
6709,6734,../../data/train\sci.electronics\53564,sci.electronics,Does anyone have some information on the relat...,,,Hard Drive Component Costs,doherty@coolpro.melpar.esys.com (Kevin Doherty),17,"E-Systems, Melpar Division","[ doherty@melpar.esys.com , doherty@melpar.es...",[+=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-...,From: doherty@coolpro.melpar.esys.com (Kevin D...,NaN,4
5461,5484,../../data/train\rec.sport.baseball\104447,rec.sport.baseball,Kevin Mitchell's sacrifice fly in the eighth o...,,,Reds snap 5-game losing streak: RedReport 4-18,rstimets@silver.ucs.indiana.edu (robert and st...,136,Indiana University,[],[---------------------------------------------...,From: rstimets@silver.ucs.indiana.edu (robert ...,NaN,4
1135,1138,../../data/train\comp.os.ms-windows.misc\9470,comp.os.ms-windows.misc,"ahh, yes, this is a fun topic. No, once the na...",Have you tried re-installing the software? Oth...,,"Re: Can I Change ""Licensed To"" Data in Windows...",2a42dubinski@vms.csd.mu.edu,19,Marquette University - Computer Services,"[ <1993Apr15.180633.3437@trintex.uucp>, , char...",[---------------------------------------------...,From: 2a42dubinski@vms.csd.mu.edu\nSubject: Re...,NaN,4
6374,6399,../../data/train\sci.crypt\15590,sci.crypt,There is a very comprehensive list in sci. mat...,,,Re: List of large integer arithmetic packages,marc@math.uni-sb.de (Marc Conrad),530,"Computational Linguistics Dept., U Saarbruecken","[mrr@scss3.cl.msu.edu , ca@math.berkeley.edu ,...",[wuarchive.wustl.edu:/edu/math/msdos/modern.al...,From: marc@math.uni-sb.de (Marc Conrad)\nSubje...,NaN,4
1125,1128,../../data/train\comp.os.ms-windows.misc\9460,comp.os.ms-windows.misc,"Hello, I'm having problems printing SPANISH ch...",,,WORD 2.0 HELP!,2a42dubinski@vms.csd.mu.edu,10,Marquette University - Computer Services,[],[---------------------------------------------...,From: 2a42dubinski@vms.csd.mu.edu\nSubject: WO...,NaN,4
1262,1265,../../data/train\comp.os.ms-windows.misc\9602,comp.os.ms-windows.misc,My understanding was that Chicago was DOS 7. D...,"Anyway, don't expect it soon. Windows 4 and DO...",,Re: RE: Win NT - what is it???,drohand@cad.gmeds.com (Dominic Drohan),16,EDS/Cadillac,[ drohand@cad.gmeds.com ],[+--------------------------------------------...,From: drohand@cad.gmeds.com (Dominic Drohan)\n...,NaN,4
1263,1266,../../data/train\comp.os.ms-windows.misc\9603,comp.os.ms-windows.misc,Hello everybody... Are there any ftp-sites wit...,,,FTP-site for .WAV-files,frode@zevs.ifi.unit.no (Frode Kvam),27,"Institutt for informatikk, Universitetet i Tr....",[ frode@ifi.unit.no ],[_____________________________________________...,From: frode@zevs.ifi.unit.no (Frode Kvam)\nSub...,NaN,4
1976,1990,../../data/train\comp.sys.ibm.pc.hardware\60434,comp.sys.ibm.pc.hardware,I have an old tandon type modem . Does anyone ...,,,Upgrading a modem ...,cdw@dcs.ed.ac.uk (Chris Walton),19,"Department of Computer Science, Edinburgh Univ...","[ cdw@dcs.ed.ac.uk. , cdw@dcs.ed.ac.uk , C.W...",[=============================================...,From: cdw@dcs.ed.ac.uk (Chris Walton)\nSubject...,NaN,4
3456,3476,../../data/train\misc.forsale\74782,misc.forsale,I have a few things to sell. All items are in ...,,,"DJ Equipment, NES Carts, Cordless Phone FOR SA...",hickson@pop.psu.edu (Darryl Hickson),35,Penn State Population Research Institute,[],[=============================================...,From: hickson@pop.psu.edu (Darryl Hickson)\nSu...,NaN,4


In [21]:
a = train['long_string'].fillna(value='').apply(lambda x: len(x))
b = a
c = b[b>4]
check_list = c.index.tolist()[-50:]
train[train['global_index'].isin(check_list)]['long_string'].tolist()

[[],
 [],
 [],
 [],
 [],
 [],
 ["!S!L3!!UU#!QJS+UT!!QJSUD#TUUQCQ3!!!UCFJ!!%c4ACSL'D)L)D!#!!)#!!",
  'eUNG1\'mSAP#mR`i-1K`l[DiNq\'MQjZA!6V`%S!eJ#GS!Q!!QqD#2P!M49m9IdHhm2frUq2Ek))G3e"Vi)+rQJ',
  'C[`%m#+E&0jf"YI2ql`VI&0qHH!R[339`\'9hY46)TR+ZkXI!pQRQKCU3%ed9R&Cr',
  'IEjf8S+#9`a6[P8p0ZC&6H0ajcY1BR"JDM3`F%lJ1&5bI+SC2Jh4Qb9C5Ke1h%V-kiaRA-NTa`b9jkRq0r[IcrC`[c!Krd3FrcHYPkh!#HN!R6JJXb5f\'b!clJkfb121qGm2MclEe,S'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['-----------------------------------------------------------------------',
  '-----------------------------------------------------------------------',
  '----------------------------------------------------------------------'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['+------------------------------------------------------------+',
  '+------------------------------------------------------------+'],
 [],
 []]

In [11]:
t = train[train["reply"].apply(lambda x: "In article" in x)]
t.groupby("label").size()

label
alt.atheism                 32
comp.graphics                4
comp.os.ms-windows.misc     10
comp.sys.ibm.pc.hardware     8
comp.sys.mac.hardware       25
comp.windows.x               4
misc.forsale                 2
rec.autos                   14
rec.motorcycles             15
rec.sport.baseball          26
rec.sport.hockey             6
sci.crypt                    4
sci.electronics             11
sci.med                     22
sci.space                   16
soc.religion.christian      14
talk.politics.guns          23
talk.politics.mideast       11
talk.politics.misc          29
talk.religion.misc          29
dtype: int64

In [12]:
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Input, Flatten, Concatenate
from tensorflow.keras.models import Model
import gensim.downloader as api
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from gensim.models import word2vec


def remove_stopwords(sent, stopwords):
    tokens = word_tokenize(sent)
    sent = " ".join([word for word in tokens if not word in stopwords])
    return sent


def eda_MAX_NB_WORDS(corpus, remove_stop=False, ratio=0.95, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', char_level=False):
    '''
    Input: data series
    '''
    #
    if remove_stop:
        en_stopwords = stopwords.words('english')
        corpus = corpus.apply(lambda x: remove_stopwords(x, en_stopwords))
        
    tokenizer_eda = Tokenizer(num_words=None, filters=filters, lower=True, char_level=char_level)  # 如果有这个, NLTK的preprocessing可以不用做
    tokenizer_eda.fit_on_texts(corpus)
    b = pd.DataFrame(tokenizer_eda.word_counts.items(), columns=['word', 'count'])
    
    
    ## by nltk
#     tfidf_vect = TfidfVectorizer(stop_words="english", norm=None, min_df=0, max_df=0.999, use_idf=False, smooth_idf=False)
#     dtm = tfidf_vect.fit_transform(corpus)
#     b = pd.DataFrame.from_dict(tfidf_vect.vocabulary_, orient='index').reset_index()
#     b.columns =['word', 'count']
    ### grouping
    
    a = b.sort_values(by='count', ascending=False).reset_index()  # 排序重建index 就是 tokenizer中的word_index

    # ########### 累加百分比 可视化
    plt.figure(figsize=(20, 5))
    word_distribution = a['count'].cumsum() / a['count'].sum()  # 求累加百分比
    word_distribution.plot()  # 出图

    # cut_index = np.argmin(abs(word_distribution-ratio)) # 找到离0.8最近的index位置
    diff = abs(word_distribution - ratio)
    cut_index = diff[diff == min(diff)].index[0]

    plt.plot([cut_index, cut_index], [0, ratio])  # 找出 固定 ratio 的index
    plt.plot([0, cut_index], [ratio, ratio])
    plt.xlabel("word_index")  # 需要先sort, 才能说是index of words.
    plt.ylabel("word_cum_counts_perc")
    plt.title("MAX_NB_WORDS Cumsum Percentage")
    plt.show()

    # ############  大概取词范围 可视化
    plt.figure(figsize=(20, 5))
    b.iloc[:, 1].plot()  # 出图
    plt.plot([cut_index, cut_index], [0, max(b['count'])])  # 找出 固定 ratio 的index
    plt.plot([0, cut_index], [max(b['count']), max(b['count'])])
    plt.xlabel("word_index")  # 需要先sort, 才能说是index of words.
    plt.ylabel("word_count")
    plt.title("MAX_NB_WORDS Percentage")
    plt.show()
    print("Cut index with", ratio * 100, "% of corpus: ", cut_index, '\n')
    # stopwords?
    print(a.sort_values(by='count', ascending=False).head(20))

    return int(cut_index)
    # return int(cut_index)+1


def eda_MAX_DOC_LEN(corpus, remove_stop=False, ratio=0.9, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', char_level=False):
    '''
    Input: list of sentences (string type)
    '''
    if remove_stop:
        en_stopwords = stopwords.words('english')
        corpus = corpus.apply(lambda x: remove_stopwords(x, en_stopwords))
        
    ## by keras
    tokenizer_eda = Tokenizer(num_words=None, filters=filters, lower=True, char_level=char_level)
    tokenizer_eda.fit_on_texts(corpus)
    dt_q1 = pd.DataFrame([len(i) for i in tokenizer_eda.texts_to_sequences(corpus)], columns=['length'])
    
    ### grouping
    c = dt_q1['length'].value_counts().sort_index()  # 频数统计, 且按index重新排序
    sent_cdf = c.cumsum() / c.sum()
    sent_pdf = c / c.sum()
    # cut_index = np.argmin(abs(sent_cdf - ratio))  # 找到离0.8最近的index位置
    diff = abs(sent_cdf - ratio)
    cut_index = diff[diff == min(diff)].index[0]

    # ########### 累加百分比 可视化
    plt.figure(figsize=(20, 5))
    sent_cdf.plot()  # 出图

    plt.plot([cut_index, cut_index], [0, ratio])  # 找出 固定 ratio 的index
    plt.plot([0, cut_index], [ratio, ratio])
    plt.xlabel("DOC_length")  # 需要先sort, 才能说是index of words.
    plt.ylabel("DOC_cumsum_length_perc")
    plt.title("MAX_DOC_LEN CDF")
    plt.show()

    plt.figure(figsize=(20, 5))
    sent_pdf.plot()  # 出图
    plt.plot([cut_index, cut_index], [0, max(sent_pdf)])  # 找出 固定 ratio 的index
    plt.plot([0, cut_index], [max(sent_pdf), max(sent_pdf)])  # 横线
    plt.xlabel("DOC_length")  # 需要先sort, 才能说是index of words.
    plt.ylabel("DOC_length_perc")
    plt.title("MAX_DOC_LEN PDF")
    plt.show()

    print("Cut index with", ratio * 100, "% of corpus: ", cut_index)
    return int(cut_index)

def eda_WORD_CLOUD(corpus):
    en_stopwords = stopwords.words('english')
    tf_vect = CountVectorizer(stop_words=en_stopwords, token_pattern=r'\b\w[\']?\w*\b', min_df=10, max_df=0.95)
    a = tf_vect.fit_transform(corpus).toarray()
    sorted_freq = dict(sorted(tf_vect.vocabulary_.items(), key=lambda x:x[1], reverse=True))
#     wordcloud = WordCloud(scale=15, width=400, height=200, relative_scaling=1, prefer_horizontal=0.8, mask = None, 
#                       min_font_size=1, max_font_size=30, background_color='white', colormap='tab20_r');
    wordcloud = WordCloud(scale=30, width=400, height=200, relative_scaling=1, prefer_horizontal=0.8, mask = None, 
                      min_font_size=1, max_font_size=30, background_color='white', colormap='Purples');
    wordcloud.fit_words(sorted_freq) # 把数据输入 wordcloud 生成器
    plt.figure(figsize=(20,10)); # 图片大小
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off");
    return sorted_freq

# eda_MAX_NB_WORDS(corpus = filtered_corpus, ratio = 0.95)
# eda_MAX_DOC_LEN(corpus = filtered_corpus, ratio=0.9)
# eda_WORD_CLOUD(corpus)

In [13]:
label_dict ={}
for label, table in train.groupby("label"):
    corpus = table['reply']
    eda_MAX_NB_WORDS(corpus=corpus, remove_stop=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+'0123456789', ratio=0.95)
    eda_MAX_DOC_LEN(corpus=corpus, remove_stop=True,  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+'0123456789', ratio=0.9)
    word_freq = eda_WORD_CLOUD(corpus)
    label_dict[label] = word_freq
#     break

KeyboardInterrupt: 

<Figure size 1440x360 with 0 Axes>

In [ ]:
label_dict

In [ ]:
label_dict[label]

In [ ]:
def plot_wordcloud(s, book_id, only=False):
    wordcloud = WordCloud().generate(s)
    wordcloud = WordCloud(max_font_size=60, stopwords=stop_words, background_color='white', width=400, height=400).generate(s)
    if only is True:
        plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title('book_id: '+ str(book_id), size=30)
    if only is True:
        plt.show()

In [ ]:
i = 0
plt.figure(figsize=(30,30))
shape = 3
for book_id, tab in dt.groupby('book_id'):
    i+=1
    plt.subplot(shape,shape,i)
    s = tab['review_sentences'].tolist()
    s = " ".join(s)
    s = normalize_string(s)
    plot_wordcloud(s, book_id)
    if i >= shape**2:
        break
    
plt.show()